In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
import json
import seaborn as sns
import re
from IPython.core.display import HTML

In [2]:
tybee = pd.read_csv('../data/tybee_may6_vrbo.csv')

In [3]:
tybee = tybee.drop_duplicates('data.results.listings.listingId')

In [4]:
tybee = tybee.rename(columns={'data.results.listings.bedrooms':'bedrooms', 'data.results.listings.propertyType':'type',
       'data.results.listings.sleeps':'sleeps', 'data.results.listings.petsAllowed':'pets_allowed',
       'data.results.listings.averageRating':'rating',
       'data.results.listings.reviewCount':'review_count',
       'data.results.listings.bathrooms.full':'full_bathrooms',
       'data.results.listings.bathrooms.half':'half_bathrooms',
        'data.results.listings.prices.perNight.roundedFormattedAmount':'cost_night',
       'data.results.listings.geoCode.latitude':'latitude',
       'data.results.listings.geoCode.longitude':'longitude'})

In [5]:
tybee

,data.results.listings.virtualTourBadge,data.results.listings.listingId,data.results.listings.detailPageUrl,data.results.listings.instantBookable,data.results.listings.listingNumber,data.results.listings.propertyId,bedrooms,type,sleeps,pets_allowed,...,data.results.filterGroups[0].filters[0].filter.refineByQueryArgument,data.results.filterGroups[0].filters[0].filter.description,data.results.filterGroups[0].filters[0].filter.__typename,data.results.filterGroups[0].filters[0].filter.groupId,errors[0].message,errors[0].extensions.errorCode,errors[0].extensions.unit,errors[0].extensions.code,errors[0].locations[0].line,errors[0].locations[0].column
0,NaN,321.2111464.2675998,/2111464?unitId=2675998&adultsCount=4&arrival=...,True,2111464,2111464,2,House,6,True,...,filter:74,NaN,Filter,13,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,321.2067496.2632028,/2067496?unitId=2632028&adultsCount=4&arrival=...,True,2067496,2067496,1,Condo,4,False,...,filter:74,NaN,Filter,13,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,121.4791378.5779882,/4791378ha?unitId=5779882&adultsCount=4&arriva...,True,4791378,4791378ha,1,Condo,4,False,...,filter:74,NaN,Filter,13,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,321.2278415.2843297,/2278415?unitId=2843297&adultsCount=4&arrival=...,True,2278415,2278415,1,Condo,4,False,...,filter:74,NaN,Filter,13,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,121.9466565.8545787,/9466565ha?unitId=8545787&adultsCount=4&arriva...,False,9466565,9466565ha,1,Condo,4,False,...,filter:74,NaN,Filter,13,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,NaN,321.2715298.3286122,/2715298?unitId=3286122&adultsCount=4&arrival=...,False,2715298,2715298,2,Resort,8,False,...,filter:74,NaN,Filter,13,NaN,NaN,NaN,NaN,NaN,NaN
444,NaN,321.2000957.2565474,/2000957?unitId=2565474&adultsCount=4&arrival=...,False,2000957,2000957,2,Resort,6,False,...,filter:74,NaN,Filter,13,NaN,NaN,NaN,NaN,NaN,NaN
445,NaN,321.2350172.2916572,/2350172?unitId=2916572&adultsCount=4&arrival=...,False,2350172,2350172,2,Villa,8,False,...,filter:74,NaN,Filter,13,NaN,NaN,NaN,NaN,NaN,NaN
446,NaN,321.3286738.3859834,/3286738?unitId=3859834&adultsCount=4&arrival=...,True,3286738,3286738,6,House,13,False,...,filter:74,NaN,Filter,13,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
tybee = tybee.drop(columns = {'data.results.listings.virtualTourBadge',
       'data.results.listings.listingId',
       'data.results.listings.detailPageUrl',
       'data.results.listings.instantBookable',
       'data.results.listings.listingNumber',
       'data.results.listings.propertyId',
        'data.results.listings.__typename',
        'data.results.listings.minStayRange.minStayHigh',
       'data.results.listings.minStayRange.minStayLow',
       'data.results.listings.minStayRange.__typename',
       'data.results.listings.propertyMetadata.headline',
       'data.results.listings.propertyMetadata.__typename',
       'data.results.listings.unitMetadata.unitName',
       'data.results.listings.unitMetadata.__typename',
        'data.results.listings.bathrooms.toiletOnly',
       'data.results.listings.bathrooms.__typename',
       'data.results.listings.spaces.__typename',
       'data.results.listings.spaces.spacesSummary.bedCountDisplay',
       'data.results.listings.spaces.spacesSummary.__typename',
       'data.results.listings.spaces.spacesSummary.area.areaValue',
       'data.results.listings.spaces.spacesSummary.area.__typename',
       'data.results.listings.geoDistance.text',
       'data.results.listings.geoDistance.relationType',
       'data.results.listings.geoDistance.__typename',
       'data.results.listings.priceSummary.priceAccurate',
       'data.results.listings.priceSummary.priceTypeId',
       'data.results.listings.priceSummary.edapEventJson',
       'data.results.listings.priceSummary.formattedAmount',
       'data.results.listings.priceSummary.roundedFormattedAmount',
       'data.results.listings.priceSummary.pricePeriodDescription',
       'data.results.listings.priceSummary.__typename',
       'data.results.listings.priceSummarySecondary.priceTypeId',
       'data.results.listings.priceSummarySecondary.edapEventJson',
       'data.results.listings.priceSummarySecondary.formattedAmount',
       'data.results.listings.priceSummarySecondary.roundedFormattedAmount',
       'data.results.listings.priceSummarySecondary.pricePeriodDescription',
       'data.results.listings.priceSummarySecondary.__typename',
       'data.results.listings.priceLabel.priceTypeId',
       'data.results.listings.priceLabel.pricePeriodDescription',
       'data.results.listings.priceLabel.__typename',
       'data.results.listings.prices.label',
       'data.results.listings.prices.mainPrice',
       'data.results.listings.prices.__typename',
       'data.results.listings.prices.perNight.amount',
       'data.results.listings.prices.perNight.formattedAmount',
         'data.results.listings.prices.perNight.pricePeriodDescription',
       'data.results.listings.prices.perNight.__typename',
       'data.results.listings.prices.total.amount',
       'data.results.listings.prices.total.formattedAmount',
       'data.results.listings.prices.total.roundedFormattedAmount',
       'data.results.listings.prices.total.pricePeriodDescription',
       'data.results.listings.prices.total.__typename',
        'data.results.listings.geoCode.__typename',
       'data.results.listings.amenitiesBadges[0].id',
       'data.results.listings.amenitiesBadges[0].name',
       'data.results.listings.amenitiesBadges[0].helpText',
       'data.results.listings.amenitiesBadges[0].__typename',
       'data.results.listings.images[0].altText',
       'data.results.listings.images[0].c6_uri',
       'data.results.listings.images[0].c9_uri',
       'data.results.listings.images[0].mab',
       'data.results.listings.images[0].__typename',
       'data.results.listings.rankedBadges[0].id',
       'data.results.listings.rankedBadges[0].name',
       'data.results.listings.rankedBadges[0].helpText',
       'data.results.listings.rankedBadges[0].__typename',
       'data.results.listings.superlativesBadges[0].id',
       'data.results.listings.superlativesBadges[0].name',
       'data.results.listings.superlativesBadges[0].helpText',
       'data.results.listings.superlativesBadges[0].__typename',
       'data.results.listings.virtualTourBadge.id',
       'data.results.listings.virtualTourBadge.name',
       'data.results.listings.virtualTourBadge.helpText',
       'data.results.listings.virtualTourBadge.__typename',
       'data.results.listings.images[0].mab.banditId',
       'data.results.listings.images[0].mab.payloadId',
       'data.results.listings.images[0].mab.campaignId',
       'data.results.listings.images[0].mab.cached',
       'data.results.listings.images[0].mab.arm',
       'data.results.listings.images[0].mab.__typename',
       'data.results.listings.priceSummary',
       'data.results.listings.priceSummarySecondary',
       'data.results.listings.priceLabel', 'data.results.listings.prices',
       'data.results.listings.webRatingBadges[0].id',
       'data.results.listings.webRatingBadges[0].name',
       'data.results.listings.webRatingBadges[0].helpText',
       'data.results.listings.webRatingBadges[0].__typename',
        'data.results.listings.prices.total', 'data.results.id',
       'data.results.propertyRedirectUrl', 'data.results.__typename',
       'data.results.resultCount', 'data.results.page',
       'data.results.pageSize', 'data.results.pageCount',
       'data.results.queryUUID', 'data.results.pinnedListing',
       'data.results.fromRecord', 'data.results.toRecord',
       'data.results.internalTools', 'data.results.typeaheadSuggestion.name',
       'data.results.typeaheadSuggestion.uuid',
       'data.results.typeaheadSuggestion.term',
       'data.results.typeaheadSuggestion.__typename',
       'data.results.geography.name', 'data.results.geography.lbsId',
       'data.results.geography.gaiaId', 'data.results.geography.isGeocoded',
       'data.results.geography.shouldShowMapCentralPin',
       'data.results.geography.__typename',
       'data.results.geography.description',
       'data.results.geography.primaryGeoType',
       'data.results.geography.location.latitude',
       'data.results.geography.location.longitude',
       'data.results.geography.location.__typename',
       'data.results.geography.breadcrumbs[0].name',
       'data.results.geography.breadcrumbs[0].countryCode',
       'data.results.geography.breadcrumbs[0].primaryGeoType',
       'data.results.geography.breadcrumbs[0].__typename',
       'data.results.geography.breadcrumbs[0].location.latitude',
       'data.results.geography.breadcrumbs[0].location.longitude',
       'data.results.geography.breadcrumbs[0].location.__typename',
       'data.results.destination.__typename',
       'data.results.destination.breadcrumbs[0].name',
       'data.results.destination.breadcrumbs[0].url',
       'data.results.destination.breadcrumbs[0].__typename',
       'data.results.destinationMessage.iconText',
       'data.results.destinationMessage.__typename',
       'data.results.destinationMessage.iconTitleText.title',
       'data.results.destinationMessage.iconTitleText.message',
       'data.results.destinationMessage.iconTitleText.icon',
       'data.results.destinationMessage.iconTitleText.messageValueType',
       'data.results.destinationMessage.iconTitleText.link',
       'data.results.destinationMessage.iconTitleText.__typename',
       'data.results.percentBooked.currentPercentBooked',
       'data.results.percentBooked.__typename', 'data.results.parsedParams.q',
       'data.results.parsedParams.sort',
       'data.results.parsedParams.__typename',
       'data.results.parsedParams.coreFilters.adults',
       'data.results.parsedParams.coreFilters.children',
       'data.results.parsedParams.coreFilters.pets',
       'data.results.parsedParams.coreFilters.minBedrooms',
       'data.results.parsedParams.coreFilters.maxBedrooms',
       'data.results.parsedParams.coreFilters.minBathrooms',
       'data.results.parsedParams.coreFilters.maxBathrooms',
       'data.results.parsedParams.coreFilters.minNightlyPrice',
       'data.results.parsedParams.coreFilters.maxNightlyPrice',
       'data.results.parsedParams.coreFilters.minSleeps',
       'data.results.parsedParams.coreFilters.__typename',
       'data.results.parsedParams.dates.arrivalDate',
       'data.results.parsedParams.dates.departureDate',
       'data.results.parsedParams.dates.__typename',
       'data.results.mapViewport.neLat', 'data.results.mapViewport.neLong',
       'data.results.mapViewport.swLat', 'data.results.mapViewport.swLong',
       'data.results.mapViewport.__typename',
       'data.results.globalMessages.alert',
       'data.results.globalMessages.__typename',
       'data.results.globalMessages.banner',
       'data.results.filterGroups[0].__typename',
       'data.results.filterGroups[0].groupInfo.id',
       'data.results.filterGroups[0].groupInfo.name',
       'data.results.filterGroups[0].groupInfo.__typename',
       'data.results.filterGroups[0].filters[0].count',
       'data.results.filterGroups[0].filters[0].checked',
       'data.results.filterGroups[0].filters[0].__typename',
       'data.results.filterGroups[0].filters[0].filter.id',
       'data.results.filterGroups[0].filters[0].filter.name',
       'data.results.filterGroups[0].filters[0].filter.refineByQueryArgument',
       'data.results.filterGroups[0].filters[0].filter.description',
       'data.results.filterGroups[0].filters[0].filter.__typename',
       'data.results.filterGroups[0].filters[0].filter.groupId',
       'errors[0].message', 'errors[0].extensions.errorCode',
        'errors[0].extensions.unit', 'errors[0].extensions.code',
       'errors[0].locations[0].line', 'errors[0].locations[0].column'})

In [7]:
tybee['cost_night'] = tybee['cost_night'].str.replace('$', '')

C:\Users\bswil\AppData\Local\Temp\ipykernel_20756\3327514090.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tybee['cost_night'] = tybee['cost_night'].str.replace('$', '')


In [8]:
tybee['cost_night'] = tybee['cost_night'].str.replace(',', '')

In [9]:
tybee = tybee.drop(265)

In [10]:
tybee = tybee.drop(282)

In [11]:
tybee['cost_night'] = tybee['cost_night'].astype(int)

In [12]:
tybee['half_bathrooms'] = tybee['half_bathrooms']/2

In [13]:
tybee['bathrooms'] = tybee['full_bathrooms']+tybee['half_bathrooms']

In [14]:
tybee = tybee.drop(columns={'full_bathrooms', 'half_bathrooms'})

In [15]:
tybee['checkin_date'] = 'May 6'

In [16]:
tybee['checkin_month'] = '5'
tybee['checkin_day'] = '6'